In [9]:
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub 
import tensorflow_datasets as tfds
import tf_keras as keras
import datetime

In [2]:
IMAGE_RES = 224

def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    return image, label

In [3]:
(raw_train, raw_validation, raw_test) , metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

print(num_examples)
print(num_classes)

23262
2


2025-02-03 09:57:16.561894: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
BATCH_SIZE = 32

train_batches = raw_train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_test.map(format_image).batch(1)

for  image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

2025-02-03 09:57:20.457336: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-02-03 09:57:22.336394: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


TensorShape([32, 224, 224, 3])

In [5]:
module_selection = ("mobilenet_v2_100_224", 224, 1280)
handle_base, pixels, FV_SIZE = module_selection

# Use the correct non-preview URL
MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4".format(handle_base)

IMAGE_SIZE = (pixels, pixels)

feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape=(224, 224, 3),
    trainable=False
)

In [6]:
model = keras.Sequential([
    feature_extractor,
    keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2562 (10.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [7]:
# compiling the model 
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
# Define the log directory for TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Define the TensorBoard callback
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)

In [18]:
# training the model 
hist = model.fit(
    train_batches, 
    epochs=5,
    validation_data=validation_batches,
    callbacks=[tensorboard_callback]
)

Epoch 1/5


582/582 [==============================] - 136s 227ms/step - loss: 0.0535 - accuracy: 0.9808 - val_loss: 0.0406 - val_accuracy: 0.9854
Epoch 2/5
582/582 [==============================] - 133s 226ms/step - loss: 0.0298 - accuracy: 0.9895 - val_loss: 0.0357 - val_accuracy: 0.9880
Epoch 3/5
582/582 [==============================] - 133s 226ms/step - loss: 0.0254 - accuracy: 0.9910 - val_loss: 0.0452 - val_accuracy: 0.9845
Epoch 4/5
582/582 [==============================] - 140s 238ms/step - loss: 0.0216 - accuracy: 0.9923 - val_loss: 0.0386 - val_accuracy: 0.9875
Epoch 5/5
582/582 [==============================] - 139s 236ms/step - loss: 0.0185 - accuracy: 0.9939 - val_loss: 0.0434 - val_accuracy: 0.9867


In [23]:
!tensorboard --logdir logs/fit

2025-02-03 10:14:24.626409: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 10:14:24.638472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738577664.652342   45421 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738577664.656182   45421 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 10:14:24.669348: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

2025-02-03 10:14:26.866981: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)


In [24]:
# evaluating the model
model.evaluate(test_batches)

2326/2326 [==============================] - 43s 18ms/step - loss: 0.0332 - accuracy: 0.9888


[0.033208198845386505, 0.9888219833374023]

In [25]:
# saving the model
CATS_VS_DOGS_SAVED_MODEL = "exp_saved_model"
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)

INFO:tensorflow:Assets written to: exp_saved_model/assets


INFO:tensorflow:Assets written to: exp_saved_model/assets


### Convert the Model to LiteRT 

In [26]:
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
lite_model = converter.convert()
lite_model_file = 'converted_model.tflite'

with open(lite_model_file, 'wb') as f:
    f.write(lite_model)

W0000 00:00:1738577943.958512   27531 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1738577943.958874   27531 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-03 10:19:03.959796: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: exp_saved_model
2025-02-03 10:19:03.974557: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-02-03 10:19:03.974600: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: exp_saved_model
I0000 00:00:1738577944.040027   27531 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-02-03 10:19:04.052772: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-02-03 10:19:04.392584: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: exp_saved_model
2025-02-03 10:19:04.529373: I tensorflow/cc/saved_model/loader.cc:466] SavedModel loa